
# Session 2 – **Exploratory Data Analysis (EDA)**

This notebook guides you through a practical, end‑to‑end EDA workflow.  
It is a warm up, after you finished it successfully, repeat the process for your own data and work through the steps.

**What you'll do**
1. Load and inspect your dataset
2. Summarize numeric & categorical variables
3. Explore missingness and duplicates
4. Visualize distributions, relationships, and correlations
5. Capture insights in short Markdown notes
6. Draft 2–3 KPI ideas informed by your EDA

**Note:** This notebook intentionally focuses on *EDA only*.  
> Cleaning/feature engineering strategies (e.g., imputation/outlier handling) will be done in the next session.


You might need to download files from:
https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/GTNEJD&version=1.2


## 0) Setup
- Update `DATA_PATH` to your chosen dataset (CSV).  
- If the file is not found, a small **synthetic demo dataset** will be generated so you can still run the notebook end‑to‑end.


In [ ]:

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Display options
pd.set_option("display.max_columns", 300)
pd.set_option("display.width", 120)

# ---- Path to your CSV ----
# If you have the Harvard PeopleSuN dataset in your data folder from last session, copy it
# in the data folder of this week and set the path like:
DATA_PATH = "../data/peoplesun_hh_anon.csv"  

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)



## 1) Load Data
- Try to read the CSV at `DATA_PATH`.
- If not found, we build a small **synthetic dataset** to let you proceed.


In [ ]:

# def make_synthetic_demo(n=1000):
#     # Toy dataset with mixed types: numeric, categorical, date, and some missingness
#     dates = pd.date_range("2021-01-01", periods=n, freq="D")
#     region = np.random.choice(["North","South","East","West"], size=n, p=[0.25,0.25,0.25,0.25])
#     income = np.random.lognormal(mean=10, sigma=0.5, size=n)  # heavily skewed
#     pop_density = np.random.gamma(shape=2.0, scale=50, size=n)
#     electrified = np.random.binomial(n=1, p=np.clip(0.3 + 0.001*(pop_density) + 0.00001*(income), 0.05, 0.95))
#     # insert some missingness
#     income[np.random.choice(n, size=n//15, replace=False)] = np.nan
#     pop_density[np.random.choice(n, size=n//20, replace=False)] = np.nan
#     # a few duplicates
#     df = pd.DataFrame({
#         "date": dates,
#         "region": region,
#         "income": income,
#         "population_density": pop_density,
#         "electrified": electrified
#     })
#     df = pd.concat([df, df.sample(5, random_state=RANDOM_SEED)], ignore_index=True)
#     return df

if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH,low_memory=False)
    source_note = f"Loaded dataset from: {DATA_PATH}"
# else:
#     df = make_synthetic_demo(n=1200)
#     source_note = "Using synthetic demo dataset (file not found)."

source_note, df.head()



## 2) First Look
**Questions**
- What do rows represent? What does each column mean?
- Do we recognize numeric vs categorical vs datetime columns?


In [ ]:

print(source_note)
print("\nShape:", df.shape)
print("\nColumns:", list(df.columns))
print("\nInfo:")
print(df.info())
#display(df.head(10))


To make your EDA more interpretable:

### a) Load and inspect the codebook

If you have something like peoplesun_codebook.csv or .xlsx:

In [ ]:
codebook = pd.read_excel("../data/peoplesun_hh_odk_codebook.xlsx")
display(codebook.head())

### b) Build a quick lookup dictionary

In [ ]:
mapping = dict(zip(codebook["name"], codebook["label"]))
df_renamed = df.rename(columns=mapping)


You have now replaced the code for questions with the full question.


In [ ]:
df_renamed.head()

Note that some columns are staying as codes, these are flattened options that were made out of answers to the last question.
Example: 211. Can you list ALL of the income sources of your household?
Then the following collumns represent options 1 to 14. The options can be found in choicebook.


## 3) Data Types & Basic Hygiene (EDA-only)
- Identify numeric, categorical, datetime columns.


In [ ]:
# Identify column types
numeric_cols = df_renamed.select_dtypes(include=[np.number]).columns.tolist()
datetime_cols = df_renamed.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns.tolist()
categorical_cols = [c for c in df_renamed.columns if c not in numeric_cols + datetime_cols]

print("Numeric:", numeric_cols)
# Print a line to separate long lists
print("-" * 40)
print("Datetime:", datetime_cols)
print("-" * 40)
print("Categorical/Other:", categorical_cols)



## 4) Descriptive Statistics
Use `.describe()` for numeric and a custom summary for categorical columns.


In [ ]:

numeric_summary = df_renamed[numeric_cols].describe().T if numeric_cols else pd.DataFrame()
categorical_summary = pd.DataFrame({
    "n_unique": [df_renamed[c].nunique(dropna=True) for c in categorical_cols],
    "top_values": [df_renamed[c].value_counts(dropna=False).head(5).to_dict() for c in categorical_cols]
}, index=categorical_cols) if categorical_cols else pd.DataFrame()

display(numeric_summary.head(5))
display(categorical_summary.head(5))



## 5) Missingness Overview
- Which columns have missing values? 


In [ ]:

missing = df_renamed.isna().mean().sort_values(ascending=False)
missing = (missing * 100).round(2).rename("%missing")
display(missing.to_frame().head(30))



## 6) Duplicate Awareness
- How many duplicated rows exist?


In [ ]:

dup_count = int(df_renamed.duplicated().sum())
total = len(df_renamed)
print(f"Duplicated rows: {dup_count} / {total} ({dup_count/total*100:.2f}%)")



## 7) Distributions (Numeric)
- Histograms help reveal shape, skew, and potential outliers.
> Tip: For skewed variables (e.g., income), you can inspect both linear and log scales. We don't have any here.


In [ ]:

def plot_histograms(frame, columns, bins=30, logscale_candidates=None):
    for col in columns:
        data = frame[col].dropna()
        if data.empty:
            continue
        plt.figure(figsize=(6,4))
        plt.hist(data, bins=bins)
        plt.title(f"Histogram: {col}")
        plt.xlabel(col)
        plt.ylabel("Count")
        plt.tight_layout()
        plt.show()

plot_histograms(df_renamed, numeric_cols[1:5], bins=30)


You see that most of numerical columns are actually boleans or not relevant as histograms, we use smarter approach to visualize our columns:

In [ ]:
def is_boolish(s: pd.Series):
    # True booleans or 0/1 only (ignoring NaN)
    if s.dtype == bool:
        return True
    if np.issubdtype(s.dtype, np.number):
        uniq = pd.unique(s.dropna())
        return set(uniq).issubset({0, 1})
    # string-y 'yes/no/true/false/1/0'
    if s.dtype == object:
        vals = s.dropna().astype(str).str.lower().unique()
        return set(vals).issubset({"0","1","true","false","yes","no"})
    return False

def is_continuous_numeric(s: pd.Series, small_card_max=12):
    if not np.issubdtype(s.dropna().dtype, np.number):
        return False
    nunq = s.nunique(dropna=True)
    # treat as continuous if reasonably many distinct values
    return nunq > small_card_max

def weighted_counts(series, weights=None):
    if weights is None:
        vc = series.value_counts(dropna=False)
        return (vc / vc.sum() * 100).sort_values(ascending=False)  # percent
    dfw = pd.DataFrame({"val": series})
    dfw["_w"] = weights
    out = dfw.groupby("val", dropna=False)["_w"].sum()
    return (out / out.sum() * 100).sort_values(ascending=False)

def smart_univariate_plot(df, col, weight_col=None, bins=30, top=12, likert_order=None):
    s = df[col]
    w = df[weight_col] if weight_col and weight_col in df.columns else None

    # 1) Boolean / multi-select dummy
    if is_boolish(s):
        # Normalize different encodings to True/False
        if s.dtype != bool:
            s = s.astype(str).str.lower().isin(["1","true","yes"])
        pct = weighted_counts(s.fillna(False), weights=w)
        plt.figure(figsize=(5,3.5))
        plt.bar(pct.index.astype(str), pct.values)
        plt.ylabel("% of records")
        plt.title(f"{col} (boolean)")
        plt.tight_layout()
        plt.show()
        return

    # 2) Continuous numeric
    if is_continuous_numeric(s):
        data = s.dropna().values
        if data.size == 0: 
            return
        plt.figure(figsize=(6,4))
        plt.hist(data, bins=bins)
        plt.title(f"Histogram: {col}")
        plt.xlabel(col); plt.ylabel("Count"); plt.tight_layout(); plt.show()

        # optional log view if highly skewed (skewness > 1)
        skew = pd.Series(data).skew()
        if np.isfinite(skew) and skew > 1 and (data > 0).mean() > 0.95:
            plt.figure(figsize=(6,4))
            plt.hist(np.log1p(data[data>0]), bins=bins)
            plt.title(f"Histogram (log1p): {col}")
            plt.xlabel(f"log1p({col})"); plt.ylabel("Count"); plt.tight_layout(); plt.show()
        return

    # 3) Small-cardinality numeric (coded categories) OR text/object
    if likert_order is not None and set(pd.unique(s.dropna())) <= set(likert_order):
        # ordered bars for Likert (e.g., [1,2,3,4,5])
        ordered = pd.Categorical(s, categories=likert_order, ordered=True)
        pct = weighted_counts(ordered, weights=w)
    else:
        pct = weighted_counts(s.astype("object"), weights=w)

    pct = pct.head(top)
    plt.figure(figsize=(7,4))
    plt.bar([str(i) for i in pct.index], pct.values)
    plt.xticks(rotation=35, ha="right")
    plt.ylabel("% of records")
    plt.title(f"{col} (top {top})")
    plt.tight_layout()
    plt.show()

def plot_eda_overview(df, cols=None, weight_col=None, bins=30, top=12):
    """Plot a sensible chart per column; if cols=None, sample a few from each type."""
    if cols is None:
        # Heuristics to pick a manageable subset automatically
        boolish = [c for c in df.columns if is_boolish(df[c])]
        continuous = [c for c in df.columns if is_continuous_numeric(df[c])]
        smallcat = [c for c in df.columns 
                    if (not is_boolish(df[c])) and (not is_continuous_numeric(df[c]))]

        cols = []
        cols += boolish[:6]
        cols += continuous[:6]
        cols += smallcat[:8]

    for c in cols:
        try:
            smart_univariate_plot(df, c, weight_col=weight_col, bins=bins, top=top)
        except Exception as e:
            print(f"[skip] {c}: {e}")


### Now plot using the above defined functions

In [ ]:
plot_eda_overview(df_renamed, weight_col="natweight", bins=30, top=12)

### Or explicitly pick columns you care about:


In [ ]:
cols = [
    "106. What is your age?",
]
plot_eda_overview(df_renamed, cols=cols, weight_col="natweight")



## 8) Numeric vs Categorical
- Compare distributions of a numeric variable **across categories** (group summaries).


In [ ]:

def grouped_summary(frame, num_col, cat_col):
    g = frame[[num_col, cat_col]].dropna().groupby(cat_col)[num_col]
    summary = pd.DataFrame({
        "count": g.count(),
        "mean": g.mean(),
        "median": g.median(),
        "std": g.std()
    }).sort_values("mean", ascending=False)
    return summary

# Print grouped summaries for the first few combinations
for cat in categorical_cols[:2]:
    for num in numeric_cols[:3]:
        print(f"\n--- {num} by {cat} ---")
        display(grouped_summary(df_renamed, num, cat).head(10))



## 9) Correlation Matrix (for continious numeric Only)
- Use with care: correlation ≠ causation.
- Still, helpful for spotting redundant features or strong linear relationships.


In [ ]:
# Filter only continuous numeric columns from your existing numeric_cols list
cont_cols = [c for c in numeric_cols if is_continuous_numeric(df_renamed[c])]
cont_cols = cont_cols[:10]  # keep the first 5 for visualization

if len(cont_cols) >= 2:
    corr = df_renamed[cont_cols].corr(numeric_only=True)
    plt.figure(figsize=(6,5))
    plt.imshow(corr, interpolation="nearest")
    plt.title("Correlation Heatmap")
    plt.colorbar()
    # Optional labels:
    # ticks = range(len(corr.columns))
    # plt.xticks(ticks, corr.columns, rotation=90)
    # plt.yticks(ticks, corr.columns)
    plt.tight_layout()
    plt.show()
    display(corr.round(3))
else:
    print("Not enough continuous numeric columns to correlate.")

You could also experiment with seaborn sns.pairplot() to see nice visualisations that match two columns.

In [ ]:
import seaborn as sns
cont_cols = [c for c in numeric_cols if is_continuous_numeric(df_renamed[c])]

# Create the pair plot
g = sns.pairplot(df_renamed[cont_cols[:5]], diag_kind="kde")

for ax in g.axes.flatten():
    # ticks
    ax.tick_params(axis="x", labelrotation=90)
    ax.tick_params(axis="y", labelrotation=0)

    # axis titles (labels)
    ax.set_xlabel("")
    #ax.set_ylabel(ax.get_ylabel(), rotation=45,  ha="right", va="center", labelpad=10)
    ax.set_ylabel("")

g.tight_layout()
plt.show()



## 10) Time Trends
- If you had a datetime column which we don't in the PeopleSuN data: Aggregate by day/week/month and plot trends.


In [ ]:

def plot_time_series(frame, date_col, value_col, freq="M"):
    sub = frame[[date_col, value_col]].dropna().copy()
    if sub.empty:
        print(f"No data to plot for {value_col}.")
        return
    sub = sub.set_index(date_col).sort_index().resample(freq).mean()
    plt.figure(figsize=(7,4))
    plt.plot(sub.index, sub[value_col])
    plt.title(f"Time Trend ({value_col}, resampled {freq})")
    plt.xlabel("Date")
    plt.ylabel(value_col)
    plt.tight_layout()
    plt.show()

if datetime_cols and numeric_cols:
    date_col = datetime_cols[0]
    for v in numeric_cols[:3]:
        plot_time_series(df_renamed, date_col, v, freq="M")



## 11) Geographic Hints
If you have **region/country** columns, consider:  
- Choropleth (later in Streamlit), or  
- Per‑region summary tables now.


In [ ]:
# 1) keep only continuous numeric columns (not coded ints / dummies)
cont_cols = [c for c in numeric_cols if is_continuous_numeric(df_renamed[c])]
geo_like = [c for c in df_renamed.columns if any(k in c.lower() for k in ["zone","lga","eaid"])]
for geo in geo_like:
    for num in cont_cols[:2]:
        print(f"\nAverage {num} by {geo}:")
        display(df_renamed.groupby(geo, dropna=False)[num].mean().sort_values(ascending=True).to_frame().head(5))



## 12) Write Down 2–3 Insights (Markdown)
Use this cell to note **what surprised you**, **potential data issues**, and **early hypotheses**.

- Insight 1: …  
- Insight 2: …  
- Hypothesis / Next question: …  



## 13) KPI Drafts (Driven by EDA)
Fill in a first draft of KPIs **based on variables that are present and meaningful** in your dataset.

| KPI Name | Definition | Formula (words) | Python Expression (sketch) |
|---|---|---|---|
| Example: Electrification Rate | % of electrified households | electrified / total_households | `df['electrified'] / df['total_households']` |
|  |  |  |  |
|  |  |  |  |

> Keep KPIs **simple and measurable**. We’ll refine them after cleaning/feature engineering.



## 14) What We'll Tackle Next (Feature Engineering Preview)
- Data type fixes (categorical, datetime)  
- Missing value strategies (when to impute vs. drop)  
- Outlier detection & robust statistics  
- Scaling/encoding and derived features for your KPIs  